In [1]:
from typing import Sequence

import clingo

In [2]:
def solve(programs: Sequence[str], grounding_context=None, sep:str=' '):
    ctl = clingo.Control(("--models", "0"))
    for program in programs:
        ctl.add("base", (), program)

    ctl.ground((("base", ()),), grounding_context)

    models = []

    with ctl.solve(yield_=True) as solve_handle:
        for i, model in enumerate(solve_handle):
            assert isinstance(model, clingo.Model)
            symbols = model.symbols(shown=True)
            print("Answer {}: {}{}{}{}{}".format(i + 1, "{", sep, sep.join(map(str, sorted(symbols))), sep, "}"))
            models.append(symbols)
        mode = "UNKNOWN"
        solve_result = solve_handle.get()
        if solve_result.satisfiable:
            mode = "SAT"
        elif solve_result.unsatisfiable:
            mode = "UNSAT"
        cardinality_suffix = ""
        if not solve_result.exhausted:
            cardinality_suffix = "+"
        print(mode, "{}{}".format(len(models), cardinality_suffix))

    return models

In [3]:
commodities = """

commodity("EUR").

"""
solve([commodities]);

Answer 1: { commodity("EUR") }
SAT 1


In [4]:
time = """

time(0..3).
startTime(ST) :- time(ST), not time(ST - 1).
endTime(ET) :- time(ET), not time(ET + 1).

"""
solve([time]);

Answer 1: { endTime(3) startTime(0) time(0) time(1) time(2) time(3) }
SAT 1


In [5]:
accounts = """

account_commodity_type("Alice:Aktiva:Kassa:EUR", "EUR", real).
account_commodity_type("Bob:Aktiva:Kassa:EUR", "EUR", real).
account_commodity_type("Alice:Passiva:Eigenkapital:EUR", "EUR", logical).
account_commodity_type("Bob:Passiva:Eigenkapital:EUR", "EUR", logical).
account_commodity_type("Alice:Passiva:Verbindlichkeiten:EUR", "EUR", logical).
account_commodity_type("Bob:Passiva:Verbindlichkeiten:EUR", "EUR", logical).

"""
solve([accounts]);

Answer 1: { account_commodity_type("Alice:Aktiva:Kassa:EUR","EUR",real) account_commodity_type("Alice:Passiva:Eigenkapital:EUR","EUR",logical) account_commodity_type("Alice:Passiva:Verbindlichkeiten:EUR","EUR",logical) account_commodity_type("Bob:Aktiva:Kassa:EUR","EUR",real) account_commodity_type("Bob:Passiva:Eigenkapital:EUR","EUR",logical) account_commodity_type("Bob:Passiva:Verbindlichkeiten:EUR","EUR",logical) }
SAT 1


In [6]:
init = """

obs_at(account_debits_credits_commodity(Account, 0, 0, Commodity), ST) :-
  startTime(ST),
  account_commodity_type(Account, Commodity, _Type).

"""

In [7]:
transaction = """

obs_at(account_debitChange_commodity(DebitAccount, Change, Commodity), T) :-
  time(T),
  account_commodity_type(DebitAccount, Commodity, _Type),
  Change > 0,
  Change = #sum{ Amount,CreditAccount : occ_at(debitAccount_creditAccount_amount_commodity(DebitAccount, CreditAccount, Amount, Commodity), T) }.

obs_at(account_creditChange_commodity(CreditAccount, Change, Commodity), T) :-
  time(T),
  account_commodity_type(CreditAccount, Commodity, _Type),
  Change > 0,
  Change = #sum{ Amount,DebitAccount : occ_at(debitAccount_creditAccount_amount_commodity(DebitAccount, CreditAccount, Amount, Commodity), T) }.

"""

In [8]:
inertia = """

obs_at(account_debits_credits_commodity(Account, PreviousDebits + Debits, PreviousCredits + Credits, Commodity), T+1) :-
  time(T+1),
  account_commodity_type(Account, Commodity, _Type),
  obs_at(account_debits_credits_commodity(Account, PreviousDebits, PreviousCredits, Commodity), T),
  obs_at(account_debitChange_commodity(Account, Debits, Commodity), T),
  obs_at(account_creditChange_commodity(Account, Credits, Commodity), T).

obs_at(account_debits_credits_commodity(Account, PreviousDebits, PreviousCredits + Credits, Commodity), T+1) :-
  time(T+1),
  account_commodity_type(Account, Commodity, _Type),
  obs_at(account_debits_credits_commodity(Account, PreviousDebits, PreviousCredits, Commodity), T),
  not obs_at(account_debitChange_commodity(Account, _, Commodity), T),
  obs_at(account_creditChange_commodity(Account, Credits, Commodity), T).

obs_at(account_debits_credits_commodity(Account, PreviousDebits + Debits, PreviousCredits, Commodity), T+1) :-
  time(T+1),
  account_commodity_type(Account, Commodity, _Type),
  obs_at(account_debits_credits_commodity(Account, PreviousDebits, PreviousCredits, Commodity), T),
  obs_at(account_debitChange_commodity(Account, Debits, Commodity), T),
  not obs_at(account_creditChange_commodity(Account, _, Commodity), T).

obs_at(account_debits_credits_commodity(Account, PreviousDebits, PreviousCredits, Commodity), T+1) :-
  time(T+1),
  account_commodity_type(Account, Commodity, _Type),
  obs_at(account_debits_credits_commodity(Account, PreviousDebits, PreviousCredits, Commodity), T),
  not obs_at(account_debitChange_commodity(Account, _, Commodity), T),
  not obs_at(account_creditChange_commodity(Account, _, Commodity), T).

"""
solve([inertia]);

Answer 1: {  }
SAT 1


<block>:4:3-10: info: atom does not occur in any rule head:
  time(T)

<block>:5:3-57: info: atom does not occur in any rule head:
  account_commodity_type(DebitAccount,Commodity,_Type)

<block>:7:41-143: info: atom does not occur in any rule head:
  occ_at(debitAccount_creditAccount_amount_commodity(DebitAccount,CreditAccount,Amount,Commodity),T)

<block>:10:3-10: info: atom does not occur in any rule head:
  time(T)

<block>:11:3-58: info: atom does not occur in any rule head:
  account_commodity_type(CreditAccount,Commodity,_Type)

<block>:13:40-142: info: atom does not occur in any rule head:
  occ_at(debitAccount_creditAccount_amount_commodity(DebitAccount,CreditAccount,Amount,Commodity),T)



In [9]:
instance = """
occ_at(debitAccount_creditAccount_amount_commodity("Alice:Aktiva:Kassa:EUR", "Alice:Passiva:Eigenkapital:EUR", 200, "EUR"),0).
occ_at(debitAccount_creditAccount_amount_commodity("Bob:Aktiva:Kassa:EUR", "Alice:Aktiva:Kassa:EUR", 50, "EUR"), 0).
occ_at(debitAccount_creditAccount_amount_commodity("Bob:Aktiva:Kassa:EUR", "Alice:Aktiva:Kassa:EUR", 50, "EUR"), 1).
occ_at(debitAccount_creditAccount_amount_commodity("Bob:Aktiva:Kassa:EUR", "Alice:Aktiva:Kassa:EUR", 75, "EUR"), 2).
occ_at(debitAccount_creditAccount_amount_commodity("Bob:Aktiva:Kassa:EUR", "Alice:Aktiva:Kassa:EUR", 25, "EUR"), 2).

"""
solve([instance]);

Answer 1: { occ_at(debitAccount_creditAccount_amount_commodity("Alice:Aktiva:Kassa:EUR","Alice:Passiva:Eigenkapital:EUR",200,"EUR"),0) occ_at(debitAccount_creditAccount_amount_commodity("Bob:Aktiva:Kassa:EUR","Alice:Aktiva:Kassa:EUR",25,"EUR"),2) occ_at(debitAccount_creditAccount_amount_commodity("Bob:Aktiva:Kassa:EUR","Alice:Aktiva:Kassa:EUR",50,"EUR"),0) occ_at(debitAccount_creditAccount_amount_commodity("Bob:Aktiva:Kassa:EUR","Alice:Aktiva:Kassa:EUR",50,"EUR"),1) occ_at(debitAccount_creditAccount_amount_commodity("Bob:Aktiva:Kassa:EUR","Alice:Aktiva:Kassa:EUR",75,"EUR"),2) }
SAT 1


In [10]:
solve([
    commodities,
    time,
    accounts,
    init,
    transaction,
    inertia,
    instance,
], sep='\n');

Answer 1: {
commodity("EUR")
endTime(3)
startTime(0)
time(0)
time(1)
time(2)
time(3)
obs_at(account_creditChange_commodity("Alice:Aktiva:Kassa:EUR",50,"EUR"),0)
obs_at(account_creditChange_commodity("Alice:Aktiva:Kassa:EUR",50,"EUR"),1)
obs_at(account_creditChange_commodity("Alice:Aktiva:Kassa:EUR",100,"EUR"),2)
obs_at(account_creditChange_commodity("Alice:Passiva:Eigenkapital:EUR",200,"EUR"),0)
obs_at(account_debitChange_commodity("Alice:Aktiva:Kassa:EUR",200,"EUR"),0)
obs_at(account_debitChange_commodity("Bob:Aktiva:Kassa:EUR",50,"EUR"),0)
obs_at(account_debitChange_commodity("Bob:Aktiva:Kassa:EUR",50,"EUR"),1)
obs_at(account_debitChange_commodity("Bob:Aktiva:Kassa:EUR",100,"EUR"),2)
obs_at(account_debits_credits_commodity("Alice:Aktiva:Kassa:EUR",0,0,"EUR"),0)
obs_at(account_debits_credits_commodity("Alice:Aktiva:Kassa:EUR",200,50,"EUR"),1)
obs_at(account_debits_credits_commodity("Alice:Aktiva:Kassa:EUR",200,100,"EUR"),2)
obs_at(account_debits_credits_commodity("Alice:Aktiva:Kassa:E